In [1]:
import sys
from pathlib import Path

# Add the src directory to Python path
src_path = str(Path("./").resolve().parent)
if src_path not in sys.path:
    sys.path.append(src_path)

sys.path

['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages',
 '/Users/anton/dev/MARS/correctness-model-internals']

In [2]:
from pathlib import Path
from collections import defaultdict

import torch as pt
import pandas as pd
# import seaborn as sns
import plotly.graph_objects as go

from src.classifying import (
    ActivationsHandler,
    combine_activations_handlers,
    get_correctness_direction_classifier,
    get_logistic_regression_classifier,
)

# sns.set_theme(style="whitegrid")


In [3]:
def load_activations(
    model_id,
    dataset_id,
    prompt_id,
    subset_id,
    input_type,
    layer,
    batch_ids=None,
):
    if batch_ids:
        batch_ids = [int(batch_id) for batch_id in batch_ids]

    paths = sorted(
        list(
            Path(
                f"../activations/{model_id}/{dataset_id}/{prompt_id}/{subset_id}/{input_type}/layer_{layer}"
            ).iterdir()
        ),
        key=lambda p: int(p.stem.split("_")[-1]),
    )

    activations_list, indices = [], []
    batch_size = None
    for batch_file in paths:
        batch_id = int(batch_file.stem.split("_")[-1])
        if batch_ids and batch_id not in batch_ids:
            continue

        activations = pt.load(batch_file, map_location=pt.device("cpu"))
        activations_list.append(activations)

        batch_size = activations.shape[0]

        if batch_size is None:
            batch_size = activations.shape[0]
        else:
            assert batch_size == activations.shape[0]

        indices.append(
            pd.Series(range(batch_size), name="index") + batch_id
        )


    return (
        pt.cat(activations_list, dim=0),
        pd.concat(indices, axis=0).reset_index(drop=True),
    )


def load_labels(model_id, dataset_id, prompt_id, subset_id, indices=None):
    paths = list(
        Path(f"../../evaluations/{model_id}/{dataset_id}/{prompt_id}/").iterdir()
    )
    for path in paths:
        filename = path.stem
        if subset_id != filename.split("_generations_evaluated")[0]:
            continue
        df = pd.read_csv(path)
        if indices is not None:
            df = df.iloc[indices].reset_index(drop=True)
        return df
    raise ValueError(
        f"No labels found for {model_id} {dataset_id} {prompt_id} {subset_id}"
    )


In [4]:
all_activation_exp_configs = defaultdict(list)
for model_path in Path("../../activations").iterdir():
    for dataset_path in model_path.iterdir():
        for prompt_path in dataset_path.iterdir():
            for subset_path in prompt_path.iterdir():
                for input_type_path in subset_path.iterdir():
                    for layer_path in input_type_path.iterdir():
                        all_activation_exp_configs["model_id"].append(model_path.name)
                        all_activation_exp_configs["dataset_id"].append(dataset_path.name)
                        all_activation_exp_configs["prompt_id"].append(prompt_path.name)
                        all_activation_exp_configs["subset_id"].append(subset_path.name)
                        all_activation_exp_configs["input_type"].append(input_type_path.name)
                        all_activation_exp_configs["layer"].append(int(layer_path.name.split("_")[-1]))
                        all_activation_exp_configs["path"].append(layer_path)
all_activation_exp_configs_df = pd.DataFrame(all_activation_exp_configs)
all_activation_exp_configs_df


,model_id,dataset_id,prompt_id,subset_id,input_type,layer,path
0,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,0,../activations/llama3.1_8b_chat/football_leagu...
1,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,7,../activations/llama3.1_8b_chat/football_leagu...
2,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,9,../activations/llama3.1_8b_chat/football_leagu...
3,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,17,../activations/llama3.1_8b_chat/football_leagu...
4,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,28,../activations/llama3.1_8b_chat/football_leagu...
...,...,...,...,...,...,...,...
251,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,23,../activations/llama3.1_8b_chat/birth_years_4k...
252,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,15,../activations/llama3.1_8b_chat/birth_years_4k...
253,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,12,../activations/llama3.1_8b_chat/birth_years_4k...
254,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,2,../activations/llama3.1_8b_chat/birth_years_4k...


In [5]:
# Set None to get all
MODEL_ID = "llama3.1_8b_chat"
# DATASET_ID = "gsm8k"
# PROMPT_ID = "cot_3_shot"
# SUBSET_ID = "main"
# INPUT_TYPE = "prompt_only"
DATASET_ID = None
PROMPT_ID = None
SUBSET_ID = None
INPUT_TYPE = None


activation_exp_configs_df = all_activation_exp_configs_df.copy()

if MODEL_ID != None:
    activation_exp_configs_df = activation_exp_configs_df[activation_exp_configs_df["model_id"] == MODEL_ID]
if DATASET_ID != None:
    activation_exp_configs_df = activation_exp_configs_df[activation_exp_configs_df["dataset_id"] == DATASET_ID]
if PROMPT_ID != None:
    activation_exp_configs_df = activation_exp_configs_df[activation_exp_configs_df["prompt_id"] == PROMPT_ID]
if SUBSET_ID != None:
    activation_exp_configs_df = activation_exp_configs_df[activation_exp_configs_df["subset_id"] == SUBSET_ID]
if INPUT_TYPE != None:
    activation_exp_configs_df = activation_exp_configs_df[activation_exp_configs_df["input_type"] == INPUT_TYPE]


activation_exp_configs_df

,model_id,dataset_id,prompt_id,subset_id,input_type,layer,path
0,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,0,../activations/llama3.1_8b_chat/football_leagu...
1,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,7,../activations/llama3.1_8b_chat/football_leagu...
2,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,9,../activations/llama3.1_8b_chat/football_leagu...
3,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,17,../activations/llama3.1_8b_chat/football_leagu...
4,llama3.1_8b_chat,football_leagues_1k,base,main,prompt_only,28,../activations/llama3.1_8b_chat/football_leagu...
...,...,...,...,...,...,...,...
251,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,23,../activations/llama3.1_8b_chat/birth_years_4k...
252,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,15,../activations/llama3.1_8b_chat/birth_years_4k...
253,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,12,../activations/llama3.1_8b_chat/birth_years_4k...
254,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,2,../activations/llama3.1_8b_chat/birth_years_4k...


In [8]:

res_dict = defaultdict(list)

for (model_id, dataset_id, prompt_id, subset_id, input_type), config_df in activation_exp_configs_df.groupby(["model_id", "dataset_id", "prompt_id", "subset_id", "input_type"]):
    print(f"\n{model_id=}, {dataset_id=}, {prompt_id=}, {subset_id=}, {input_type=}")
    labels_df = load_labels(
        model_id=model_id,
        dataset_id=dataset_id,
        prompt_id=prompt_id,
        subset_id=subset_id,
    )

    check_indices = None
    for layer in config_df["layer"].astype(int).sort_values():
        print(f"{layer=}", end=", ")
        activations, indices = load_activations(
            model_id=model_id,
            dataset_id=dataset_id,
            prompt_id=prompt_id,
            subset_id=subset_id,
            input_type=input_type,
            layer=layer,
        )
        
        if check_indices is None:
            check_indices = indices.sample(frac=1, replace=False)
        
        if set(indices) != set(check_indices):
            raise RuntimeError(f"indices across layers are not the same")

        labels_df_subset = labels_df.iloc[check_indices]
        activations = activations[check_indices]

        activations_handler = ActivationsHandler(
            activations=activations,
            labels=labels_df_subset["correct"].astype(bool),
        )

        activations_handler_folds = list(
            activations_handler.split_dataset(split_sizes=[1/5] * 5)
        )

        fold_stats = {}
        for fold_i, activations_handler_test in enumerate(activations_handler_folds):
            activations_handler_train = combine_activations_handlers(
                [ah for j, ah in enumerate(activations_handler_folds) if j != fold_i]
            ).sample_equally_across_groups(
                group_labels=[False, True]
            )
            activations_handler_test = activations_handler_test.sample_equally_across_groups(
                group_labels=[False, True]
            )

            res_dict["model_id"].append(model_id)
            res_dict["dataset_id"].append(dataset_id)
            res_dict["prompt_id"].append(prompt_id)
            res_dict["subset_id"].append(subset_id)
            res_dict["input_type"].append(input_type)
            res_dict["layer"].append(layer)
            res_dict["fold"].append(fold_i)

            direction_classifier, direction_calculator = get_correctness_direction_classifier(
                activations_handler_train=activations_handler_train,
                activations_handler_test=activations_handler_test,
            )
            # res_dict["classifying_direction"].append(direction_calculator.classifying_direction.tolist())
            for key, value in direction_classifier.classification_metrics.items():
                res_dict[f"direction_{key}"].append(value)
            
            for key, value in get_logistic_regression_classifier(
                    activations_handler_train=activations_handler_train,
                    activations_handler_test=activations_handler_test,
                )[0].classification_metrics.items():
                res_dict[f"logistic_regression_{key}"].append(value)



res_df = pd.DataFrame(res_dict)
res_df


model_id='llama3.1_8b_chat', dataset_id='birth_years_4k', prompt_id='base', subset_id='main', input_type='prompt_answer'
layer=0, 

/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

layer=1, 

/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

layer=2, 

...

layer=30, 

/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

layer=31, 

/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anton/dev/MARS/correctness-model-internals/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

,model_id,dataset_id,prompt_id,subset_id,input_type,layer,fold,direction_optimal_cut,direction_optimal_train_set_cut,direction_test_roc_auc,...,direction_f1_score,direction_precision_score,direction_recall_score,logistic_regression_optimal_cut,logistic_regression_optimal_train_set_cut,logistic_regression_test_roc_auc,logistic_regression_accuracy_score,logistic_regression_f1_score,logistic_regression_precision_score,logistic_regression_recall_score
0,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,0,0,-0.188253,-0.188253,0.607367,...,0.626804,0.537102,0.752475,0.5,0.528788,0.554357,0.549505,0.562500,0.546729,0.579208
1,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,0,1,0.084373,0.084373,0.578144,...,0.509915,0.552147,0.473684,0.5,0.515389,0.586676,0.560526,0.572890,0.557214,0.589474
2,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,0,2,0.064732,0.064732,0.545049,...,0.518519,0.532995,0.504808,0.5,0.510736,0.572231,0.560096,0.575406,0.556054,0.596154
3,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,0,3,0.060345,0.060345,0.529195,...,0.509485,0.534091,0.487047,0.5,0.541316,0.599855,0.577720,0.578811,0.577320,0.580311
4,llama3.1_8b_chat,birth_years_4k,base,main,prompt_answer,0,4,0.078410,0.078410,0.541298,...,0.545455,0.536313,0.554913,0.5,0.538939,0.564336,0.552023,0.570637,0.547872,0.595376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,llama3.1_8b_chat,medals_9k,base,main,prompt_only,31,0,-0.602204,-0.602204,0.633091,...,0.596364,0.639626,0.558583,0.5,0.485272,0.821632,0.771798,0.772881,0.769231,0.776567
1276,llama3.1_8b_chat,medals_9k,base,main,prompt_only,31,1,12.907405,12.907405,0.616214,...,0.416996,0.732639,0.291436,0.5,0.483651,0.821523,0.772790,0.774503,0.768707,0.780387
1277,llama3.1_8b_chat,medals_9k,base,main,prompt_only,31,2,7.063498,7.063498,0.620428,...,0.498208,0.703797,0.385576,0.5,0.450687,0.816423,0.762829,0.765432,0.757123,0.773925
1278,llama3.1_8b_chat,medals_9k,base,main,prompt_only,31,3,9.811630,9.811630,0.625418,...,0.502203,0.686747,0.395833,0.5,0.434572,0.811460,0.763889,0.766804,0.757453,0.776389


In [ ]:
res_file = "./classification_data/res_df_llama31_8B_4_memory_datasets.csv"

In [11]:
res_df.to_csv(res_file, index=False)

In [ ]:
res_df = res_file.read_csv(res_file)

In [49]:
def plot_interactive_lineplot(df_dict, x_label, y_label, title=None, save_path=None):
    """
    df_dict: Dictionary mapping labels to dataframes, where each dataframe contains multiple columns
            for the same measurement (e.g., {'Metric 1': df1, 'Metric 2': df2})
    """
    fig = go.Figure()
    
    # Define a color palette for different metrics
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    
    for metric_idx, (metric_name, df) in enumerate(df_dict.items()):
        color = colors[metric_idx % len(colors)]
        
        # Calculate statistics for each column
        means = df.mean(axis=1)

        # Add mean line
        fig.add_trace(go.Scatter(
            x=df.index,
            y=means,
            mode='lines+markers',
            line=dict(color=color, width=2),
            marker=dict(size=8),
            name=f'{metric_name}',
            legendgroup=metric_name,
        ))

        if df.shape[1] > 1:
            stds = df.std(axis=1)
            mins = df.min(axis=1)
            maxs = df.max(axis=1)

            # Add min/max range (very faint)
            fig.add_trace(go.Scatter(
                x=df.index,
                y=maxs,
                mode='lines',
                line=dict(width=0),
                showlegend=False,
                name=f'{metric_name} Max',
                legendgroup=metric_name,
            ))
            fig.add_trace(go.Scatter(
                x=df.index,
                y=mins,
                mode='lines',
                line=dict(width=0),
                fillcolor=f'rgba{tuple(list(int(color.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + [0.1])}',
                fill='tonexty',
                showlegend=False,
                name=f'{metric_name} Min',
                legendgroup=metric_name,
            ))

            # Add ±1 std range (moderately faint)
            fig.add_trace(go.Scatter(
                x=df.index,
                y=means + stds,
                mode='lines',
                line=dict(width=0),
                showlegend=False,
                name=f'{metric_name} +1 STD',
                legendgroup=metric_name,
            ))
            fig.add_trace(go.Scatter(
                x=df.index,
                y=means - stds,
                mode='lines',
                line=dict(width=0),
                fillcolor=f'rgba{tuple(list(int(color.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + [0.3])}',
                fill='tonexty',
                showlegend=False,
                name=f'{metric_name} -1 STD',
                legendgroup=metric_name,
            ))

            # Add individual points for each fold
            for col in df.columns:
                fig.add_trace(go.Scatter(
                    x=df.index,
                    y=df[col],
                    mode='markers',
                    marker=dict(
                        color=color,
                        size=6,
                        opacity=0.5
                    ),
                    showlegend=False,
                    name=f'{metric_name} Fold {col}',
                    legendgroup=metric_name,
                ))

    fig.update_layout(
        title=title,
        yaxis_title=y_label,
        xaxis_title=x_label,
        template='plotly_dark',
        plot_bgcolor='rgba(32, 32, 32, 1)',
        paper_bgcolor='rgba(32, 32, 32, 1)',
        font=dict(color='white'),
        margin=dict(t=100, l=50, r=30, b=50),  # Increased top margin to accommodate legend
        showlegend=True,
        legend=dict(
            orientation="h",    # Horizontal legend
            yanchor="bottom",  
            y=1.02,           # Position above the plot
            xanchor="center",  # Center horizontally
            x=0.5,
            bgcolor='rgba(32, 32, 32, 0.8)'  # Semi-transparent background
        ),
        width=1000,   # Set explicit width in pixels
        height=600    # Set explicit height in pixels
    )
    
    # Update axes for consistency with dark theme
    fig.update_xaxes(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False)
    fig.update_yaxes(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False)
    
    return fig

In [ ]:
for classifier in ["direction", "logistic_regression"]:
    # for metric in ["f1_score", "accuracy_score", "precision_score", "recall_score"]:
    for metric in ["f1_score"]:
        plot_dict = {}
        for conf, res_df_ in res_df.groupby(["model_id", "dataset_id", "prompt_id", "subset_id", "input_type"]):
            # if conf[4] != "prompt_only":
            #     continue
            
            print(f"{conf=}")
            res_df_pivot = pd.pivot(
                res_df_.drop(columns=["model_id", "dataset_id", "prompt_id", "subset_id", "input_type"]),
                index='layer',
                columns='fold',
                # values=['direction_f1_score', 'logistic_regression_f1_score']  # add all metrics you want to keep
            )
            # for classifier in ["direction", "logistic_regression"]:
            #     for metric in ["f1_score", "accuracy_score", "precision_score", "recall_score"]:
            plot_dict[str(conf)] = res_df_pivot[[f"{classifier}_{metric}"]]

        plot_interactive_lineplot(
            plot_dict,
            x_label="Layer",
            y_label=f"{classifier}_{metric}".replace("_", " ").title(),
            save_path=f"./figures/{classifier}_{metric}_test.html"
        ).show()



conf=('llama3.1_8b_chat', 'birth_years_4k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'birth_years_4k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'cities_10k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'cities_10k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'football_leagues_1k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'football_leagues_1k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'medals_9k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'medals_9k', 'base', 'main', 'prompt_only')


conf=('llama3.1_8b_chat', 'birth_years_4k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'birth_years_4k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'cities_10k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'cities_10k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'football_leagues_1k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'football_leagues_1k', 'base', 'main', 'prompt_only')
conf=('llama3.1_8b_chat', 'medals_9k', 'base', 'main', 'prompt_answer')
conf=('llama3.1_8b_chat', 'medals_9k', 'base', 'main', 'prompt_only')


In [22]:
# def plot_interactive_box(df, x_label, y_label, title=None):
#     fig = go.Figure()
    
#     # Create box plot
#     fig.add_trace(go.Box(
#         x=[idx for idx in df.index for _ in range(len(df.columns))],  # Repeat each index for each fold
#         y=df.values.flatten(),  # Flatten all values
#         boxpoints='all',  # Show all points
#         jitter=0,        # No jitter for points
#         pointpos=0,      # Position points at center
#         marker=dict(
#             color='#1f77b4',
#             size=8,
#             opacity=0.5
#         ),
#         line=dict(
#             color='#1f77b4',
#             width=2
#         ),
#         fillcolor='rgba(68, 138, 255, 0.5)',
#         opacity=0.6,
#         showlegend=False,
#         boxmean=True,    # Show mean as a dashed line
#         width=0.5        # Width of boxes
#     ))

#     fig.update_layout(
#         title=title,
#         yaxis_title=y_label,
#         xaxis_title=x_label,
#         template='plotly_dark',
#         plot_bgcolor='rgba(32, 32, 32, 1)',
#         paper_bgcolor='rgba(32, 32, 32, 1)',
#         font=dict(color='white'),
#         margin=dict(t=50, l=50, r=30),
#         showlegend=False
#     )
    
#     # Update axes for consistency with dark theme
#     fig.update_xaxes(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False)
#     fig.update_yaxes(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False)
    
#     return fig

# plot_interactive_box(res_df, x_label="Layer", y_label="F1 Score").show()